# **Outil d'alignement de lieux avec les informations de Wikidata**

*Document d'entrée* : un fichier txt contenant la liste des lieux à aligner.

*Document de sortie* : un fichier JSON contenant les informations sur les lieux récoltées via Wikidata. Ce fichier sera utile pour la création de cartes de chaleur.

In [ ]:
#@markdown # Connexion du notebook à son compte Google Drive et signalement du dossier de travail :

#@markdown - Lancer la cellule
#@markdown - Cliquer sur « Exécuter malgré tout » lors de l’apparition du message d’avertissement indiquant que le notebook n’a pas été créé par Google
#@markdown - Cliquer sur « Se connecter à Google Drive » lors de l’apparition du second message d’avertissement pour donner l’autorisation au notebook d’accéder à vos fichiers Google Drive
#@markdown - Choisir son compte Gmail puis cliquer sur « Autoriser »

#Installation des biliothèques nécessaires
!pip install --upgrade wptools
!pip install pgeocode
from google.colab import drive
import urllib.parse, urllib.request
from collections import Counter
import pandas as pd
import json
import csv
import re
import glob
import os
import wptools
import itertools
import pgeocode
from pandas.plotting import table

drive.mount('/content/drive/')
%cd /content/drive/My Drive/

#@markdown ###Indiquer le chemin vers le dossier de travail sur le Google Drive (si le dossier n'existe pas, il sera créé lors du lancement de la cellule) :
chemin_vers_dossier_travail = '/content/drive/My Drive/lieux_wikidata/'#@param {type:"string"}

if not os.path.exists(chemin_vers_dossier_travail):
      os.makedirs(chemin_vers_dossier_travail)
os.chdir(chemin_vers_dossier_travail)

In [ ]:
#@markdown # Récupération des informations Wikidata des lieux contenus dans le fichier texte :

#@markdown ####Indiquer le chemin vers le fichier txt sur le Google Drive contenant la liste des lieux :
chemin_vers_fichier_texte = '/content/drive/My Drive/lieux_wikidata/liste_lieux.txt' #@param {type:"string"}

#@markdown ####Changer la langue si nécessaire (par défaut : français) :
langue = "fr" #@param {type:"string"}
#@markdown ####Si la détection des lieux  n'est pas assez précise, descendre le seuil à 0.8 (moins de noms vont être identifiés). Dans le cas contraire, monter le seuil à 1.0 :
seuil = "1.0" #@param {type:"string"}

#@markdown ####Indiquer le nom du fichier JSON créé :
nom_fichier_json = 'infos_lieux.json' #@param {type:"string"}


def CallWikifier(text, lang=langue, threshold=float(seuil)): 
  # Thresold = taux de sureté. 0.8 = sûr que le résultat est exact; mais peu de résultat.
  # 1 = tous les résultats mais certains faux résultats
    # Url de demande:
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "nqvsutgqswvfmrcvyxjtopvpiukjtp"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "4"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    url = "http://www.wikifier.org/annotate-article"
    # Appel de Wikifier 
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 100) as f:
        global response
        response = f.read()
        response = json.loads(response.decode("utf8"))


# Création des listes et dictionnaires qui vont contenir les informations 
# récupérées grâce à l'identifiant unique Wikidata.
donnees = {}
nom = []
type_lieu = []
latitude = []
longitude = []
longitude = []
description = []
document = []
region = []
code_postal = []

file = chemin_vers_fichier_texte
with open(file, "r") as g:
    lieux_corpus = g.read()

# Pour chaque lieu, appel de la fonction Callwikifier 
# Récupération des informations: type de lieu, région, code postal,
# latitude, longitude, description, document et enregistrement sous
# forme de dictionnaire

CallWikifier(text=lieux_corpus)
base = os.path.basename(file)
for annotation in response["annotations"]: ## pour chaque réponse:
    try:
      itemid = annotation["wikiDataItemId"]
      page = wptools.page(wikibase=itemid, lang="fr")
      page.get_wikidata()
      page.get_parse()
      infobox = page.data['infobox']
      if infobox is not None:
        latitude = infobox.get("latitude")
        nom = infobox.get("nom")
        type_lieu = infobox.get("type")
        if type_lieu is None: 
          type_lieu = page.data['what']
        else:
          type_lieu = infobox.get("type")
        latitude = infobox.get("latitude")
        longitude = infobox.get("longitude")
        description = page.data["description"]
        tout=page.data['wikidata']
        code_postal = tout.get("code postal (P281)")
        if code_postal is not None:
          nomi = pgeocode.Nominatim('fr')
          if type(code_postal) is list:
            info = nomi.query_postal_code(code_postal[0])
            region = str(info["state_name"])
          else: 
            info = nomi.query_postal_code(code_postal)
            region = str(info["state_name"])
        else:
          code_postal = None
      else:
        pass
      donnees[nom] = {"Type de lieu" : type_lieu, "Région": region, "Code Postal": code_postal,
                              "Latitude" : latitude, "Longitude" : longitude, "Description": description, "Document": base}
      print(donnees)
    except KeyError as guh:
      pass
                    
with open(nom_fichier_json, "a") as gjoi:
  json.dump(donnees, gjoi, indent=4, ensure_ascii=False)